In [31]:
#function for creating initial state
def initialState(nqubits):
    init = Circuit()
    for i in range(nqubits):
        init = init.x(i).h(i)
    return init


In [32]:
#applies mixer hamiltonian
def mixer(nqubits, beta):
    mixerU = Circuit()
    for i in range(nqubits):
        mixerU = mixerU.rx(i, -2*beta)
    return mixerU

In [33]:
#applies problem hamiltonian
def problem(nqubits, objective, gamma):
    problemU = Circuit()
    #diagonals
    for i in range (nqubits):
        obj = objective[i,i]
        if obj != 0:
            problemU = problemU.rz(i, -2*gamma*obj)
    
    #coupled terms
    for i in range (nqubits):
        for j in range (nqubits):
            if i < j:
                obj = objective[i,j]
                if obj != 0:
                    problemU = problemU.zz(i, j, -2*gamma*obj)
    return problemU


In [38]:
def createCirc(objective, params):
    circ = Circuit()
    nqubits = int(m.sqrt(objective.size))
    circ.add(initialState(nqubits))
    
    layers = int(len(params)/2)
    gammas = params[:layers]
    betas = params[layers:]
    
    #alternating layers of circuit
    for i in range(layers):
        circ.add(problem(nqubits, objective, gammas[i]))
        circ.add(mixer(nqubits, betas[i]))
        
    return circ

In [35]:
import numpy as np
import math as m
from scipy.optimize import minimize
from braket.circuits import Circuit, Observable, FreeParameter
from braket.aws import AwsSession, AwsDevice
from braket.devices import LocalSimulator

In [45]:
#for simulator
def getExpection(params):
    global objective
    global shots
    global maxIter
    global method
    global backend
        
    circ = createCirc(objective, params)
    result = backend.run(circ, shots = shots).result()
    counts = result.measurement_counts()
    pairs = zip(counts.keys(), counts.values())
        
    avgE = 0
    allE = 0
    for key, value in pairs:
        avgE += sumE(objective, key) * value
        allE += value
        
    #returns the expectation value of ALL the values in our solution set
    return avgE/allE

In [43]:
def sumE(objective, keys):
    variables = []
    for i in range(len(keys)):
        if keys[i] == '1':
            variables.append(i)
    sumE = 0
    for var1 in variables:
        sumE += objective[var1, var1]
        for var2 in variables:
            if var1 < var2:
                sumE += objective[var1, var2]
    return sumE

In [39]:
# create parameters
gammas = [FreeParameter('gamma_1'), FreeParameter('gamma_2')]
betas = [FreeParameter('beta_1'), FreeParameter('beta_2')]
params = gammas + betas

#objective matrix test
J_sub = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])

# get test circuit
testCirc = createCirc(J_sub, params)

# print test circuit
print('Printing test circuit:')
print(testCirc)

Printing test circuit:
T  : |0|1|      2       |      3       |      4       |      5       |      6       |      7       |      8      |
                                                                                                                  
q0 : -X-H-Rz(-2*gamma_1)-ZZ(-2*gamma_1)-Rx(-2*beta_1)--Rz(-2*gamma_2)-ZZ(-2*gamma_2)-Rx(-2*beta_2)----------------
                         |                                            |                                           
q1 : -X-H----------------|--------------ZZ(-2*gamma_1)-Rx(-2*beta_1)--|--------------ZZ(-2*gamma_2)-Rx(-2*beta_2)-
                         |              |                             |              |                            
q2 : -X-H----------------ZZ(-2*gamma_1)-ZZ(-2*gamma_1)-Rx(-2*beta_1)--ZZ(-2*gamma_2)-ZZ(-2*gamma_2)-Rx(-2*beta_2)-

T  : |0|1|      2       |      3       |      4       |      5       |      6       |      7       |      8      |

Unassigned parameters: [beta_1, beta_2, gamma_1, gamma_

In [ ]:
#uncomment to try simulator
# objective = J_sub
# shots = 10
# maxIter = 1
# method = 'COBYLA'
# backend = LocalSimulator()
    
# res = minimize(getExpection, params0, method = method, options = {'maxiter' = maxiter})
# res